In [4]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 987.0 kB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 18.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488513 sha256=7a8fa44ca6cf4e9cf29f1ea0f1c238e4dff0e1abd92cfc7fa2b2c1afc1dd2278
  Stored in directory: /home/kat/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [20]:
from pyspark.sql import SparkSession

mysql_connector_jar: str = "/usr/local/spark/jars/mysql-connector-j-8.2.0.jar"

In [21]:
spark = SparkSession.builder \
    .appName("MySQL") \
    .config("spark.jars", mysql_connector_jar) \
    .getOrCreate()

In [22]:
jdbc_url = "jdbc:mysql://localhost:3306/TorontoRestaurants"
connection_properties = {
    "user": "root",
    "password" : "AzurePassword",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [25]:
df = spark.read.jdbc(url=jdbc_url, table="restaurants", properties=connection_properties)

In [26]:
df.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Restaurant_Address: string (nullable = true)
 |-- Postal_Code: string (nullable = true)
 |-- Restaurant_Name: string (nullable = true)
 |-- Restaurant_Phone: string (nullable = true)
 |-- Restaurant_Price_Range: string (nullable = true)
 |-- Restaurant_Website: string (nullable = true)
 |-- Restaurant_Yelp_URL: string (nullable = true)
 |-- Restaurant_Latitude: double (nullable = true)
 |-- Restaurant_Longitude: double (nullable = true)



In [28]:
df.show(10)

+--------+--------------------+-----------+--------------------+----------------+----------------------+--------------------+--------------------+-------------------+--------------------+
|Category|  Restaurant_Address|Postal_Code|     Restaurant_Name|Restaurant_Phone|Restaurant_Price_Range|  Restaurant_Website| Restaurant_Yelp_URL|Restaurant_Latitude|Restaurant_Longitude|
+--------+--------------------+-----------+--------------------+----------------+----------------------+--------------------+--------------------+-------------------+--------------------+
|  Afghan|14 Prince Arthur ...|    M5R 1A9|            The Host|  (416) 962-4678|                $11-30|   welcometohost.com|https://www.yelp....|          43.669935|          -79.395858|
|  Afghan|259 Wellington St...|     ON M5V|Aanch Modernist I...|  (647) 558-1508|                $11-30|            aanch.ca|https://www.yelp....|          43.644708|           -79.39067|
|  Afghan|736 Bay Street\nT...|    M5G 2J8|Silk Road Kabob H

In [31]:
df_dropped = df.drop("Restaurant_Website", "Restaurant_Yelp_URL")
df_dropped.show(10)

+--------+--------------------+-----------+--------------------+----------------+----------------------+-------------------+--------------------+
|Category|  Restaurant_Address|Postal_Code|     Restaurant_Name|Restaurant_Phone|Restaurant_Price_Range|Restaurant_Latitude|Restaurant_Longitude|
+--------+--------------------+-----------+--------------------+----------------+----------------------+-------------------+--------------------+
|  Afghan|14 Prince Arthur ...|    M5R 1A9|            The Host|  (416) 962-4678|                $11-30|          43.669935|          -79.395858|
|  Afghan|259 Wellington St...|     ON M5V|Aanch Modernist I...|  (647) 558-1508|                $11-30|          43.644708|           -79.39067|
|  Afghan|736 Bay Street\nT...|    M5G 2J8|Silk Road Kabob H...|                |             Under $10|          43.659816|          -79.385591|
|  Afghan|691 Yonge Street\...|    M4Y 2B2|        Naan & Kabob|  (416) 972-6623|                $11-30|          43.669058|

In [33]:
df_dropped.write.csv("toronto_restaurants", header=True)